In [34]:
%load_ext autoreload
%autoreload 2

import jax.numpy as jnp

from compressible_1d import numerics, plot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
n_cells = 10
n_ghost_cells = 1  # per side

delta_t = 1e-4  # normalized
end_time = 1e-2  # [s]
n_steps = int(end_time / delta_t)

tube_length = 1.0  # [m]
delta_x = 1 / n_cells  # normalized cell length
gamma = 1.4

U_init = jnp.ones((3, n_cells))

U_field = U_init

U_solutions = U_init[:, :, jnp.newaxis]
for i in range(n_steps):
    U_field = numerics.step(
        U_field,
        n_ghost_cells,
        delta_x,
        delta_t,
        gamma,
    )
    U_solutions = jnp.concatenate([U_solutions, U_field[:, :, jnp.newaxis]], axis=2)

# visualize result
plot.plot_U_field(U_solutions).show(renderer="browser")

In [8]:
from compressible_1d import boundary_conditions_test

%load_ext autoreload
%autoreload 2
boundary_conditions_test.test_reflective_bc()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
